# Anonymous Challenge

Justin Ferrara

In [1]:
# Import required packages
# If packages do not load on different computer, run !pip install [package]
import os
import pandas as pd
import numpy as np
from datetime import datetime

### Prompt
A company has reached out to you for help creating their very first business metrics. You have been given a randomized dataset containing their mock transaction data. It might require cleaning some messy columns, and you may find some fields are not that useful for your analysis. 

Select the top 3 most important KPIs for this company to measure. Explain to us what they are and why they are important. Create a dashboard around these KPIs that will help the company stay on track.

### Approach

1. Business judgement
2. Data collection
3. Data preparation / cleaning
4. Data analysis
5. Data presentation

### Business judgement

The provided data contains transaction-level detail including user phone number, total spend, approximate location (zip code), timestamp of transaction, etc. Since the mock business has not established any business metrics previously, I will assume understanding historical and current revenue numbers is of high importance. Consequently, I will deconstruct revenue into its component parts to explore how the business can track, monitor, and analyze revenue to identify how revenue may be improved.

* Monthly revenue
    * Average spend per transaction
    * Number of transactions
        * Average number of transactions per user
        * Number of users
            * Number of new users
            * Number of returning users

It is important to not allow available data to limit hypothesis generation when structuring a business case like revenue because new data may be able to be collected. As such, my initial hypothesis for key KPIs may not all be possible with currently available data.

The three metrics chosen for this exercise were (1) *monthly revenue*, (2) *average spend per transaction*, and (3) *monthly transactions*. When considering KPIs for a company, it is important to choose metrics that are important indicators of overall health of the business and to adopt metrics that are mutually exclusive, but collectively exhaustive to allow for clear actionability.

In this scenario, monthly revenue was chosen as a priority KPI as it is indicative of overall business health, yet monthly revenue alone is not as actionable as it could be. By decomposing revenue into its component parts, we see that average spend per transactions and the number of transactions considered together can explain any fluctuations in total revenue. This means that if revenue, let's say, decreases next month, an analyst can look to see if the decrease is driven by lower spending or lower usage. 

These same metrics may be tracked at a different unit of aggregation as well. For example, we can assess revenue and its component parts for each individual spot.

If additional user data were made available, each metric could be broken down even further to arrive at more specific metrics. In this example, the number of transactions can be boiled down to usage by user and the number of users. And so on, as illustrated by the graphic below.

### Data collection

Transaction-level data has been provided in CSV form.

In [2]:
# Set working directory to a folder containing raw data file
os.chdir("/Users/JustinFerrara/Desktop/Anonymous")

In [3]:
raw_data = pd.read_csv('data-cEOUaHISyGUP1JLWJN8i.csv')
raw_data.head(10)

,transaction_id,created_at_UTC,user_phone,amount_paid_cents,zip_code,spot_id
0,1,2022-02-03 08:59:46,(674) 384-2776,9 065,30006,5
1,2,2021-09-02 15:55:02,1-251-843-1844,6 844,23841,5
2,3,2022-05-11 11:04:50,1-451-684-9420,9 446,482219,9
3,4,2021-01-24 17:05:12,1-658-836-8233,660,15537,8
4,5,2020-11-22 08:13:02,(481) 946-2170,5 714,221573,11
5,6,2021-08-27 16:24:06,(771) 647-7413,8 822,66352,3
6,7,2022-10-10 13:38:37,1-928-938-1403,4 526,354135,5
7,8,2022-07-18 02:22:57,1-903-517-7431,5 284,563958,8
8,9,2021-08-26 14:12:47,(774) 472-3583,3 413,41744-248,11
9,10,2021-12-17 18:35:16,(392) 515-2956,182,38402,9


In [4]:
raw_data.dtypes

transaction_id        int64
created_at_UTC       object
user_phone           object
amount_paid_cents    object
zip_code             object
spot_id               int64
dtype: object

### Data preparation / cleaning
After loading the raw CSV file and previewing the dataset, we immediately see a few things that may need attention for our analysis. Initial observations include:

* created_at_UTC is a timestamp field stored in Coordinated Universal Time (UTC) standard. This means, a local timezone is needed in order to conduct any analysis on transactions by time of day.
* The user_phone field appears to contain phone numbers of variable format. Phone numbers will need to be standardized before incorporating this field into an analysis.
* amount_paid_cents appears to use a space character possibly in place of a decimal point or a comma. Clarification is needed.
* The zip_code field is inconsistent -- some data points are in standard five character format, some include 4 digit codes to further specify location, some have anywhere from 5-6 or more digits.

Let's start by cleaning up the date-time field.

In [5]:
# Convert to datetime data type
raw_data['created_at_UTC'] = pd.to_datetime(raw_data['created_at_UTC'], format='%Y-%m-%d %H:%M:%S', errors='coerce')

# Create new fields for month, year
raw_data['year'] = pd.to_datetime(raw_data['created_at_UTC']).dt.strftime('%Y')
raw_data['month'] = pd.to_datetime(raw_data['created_at_UTC']).dt.strftime('%m')

raw_data['year_month'] = raw_data['year'] + raw_data['month']

raw_data.head(10)

,transaction_id,created_at_UTC,user_phone,amount_paid_cents,zip_code,spot_id,year,month,year_month
0,1,2022-02-03 08:59:46,(674) 384-2776,9 065,30006,5,2022,02,202202
1,2,2021-09-02 15:55:02,1-251-843-1844,6 844,23841,5,2021,09,202109
2,3,2022-05-11 11:04:50,1-451-684-9420,9 446,482219,9,2022,05,202205
3,4,2021-01-24 17:05:12,1-658-836-8233,660,15537,8,2021,01,202101
4,5,2020-11-22 08:13:02,(481) 946-2170,5 714,221573,11,2020,11,202011
5,6,2021-08-27 16:24:06,(771) 647-7413,8 822,66352,3,2021,08,202108
6,7,2022-10-10 13:38:37,1-928-938-1403,4 526,354135,5,2022,10,202210
7,8,2022-07-18 02:22:57,1-903-517-7431,5 284,563958,8,2022,07,202207
8,9,2021-08-26 14:12:47,(774) 472-3583,3 413,41744-248,11,2021,08,202108
9,10,2021-12-17 18:35:16,(392) 515-2956,182,38402,9,2021,12,202112


Next, we can standardize the user_phone field.

In [6]:
# Remove all non-digit characters join the separated character list back into one string
raw_data['user_phone_edited'] = raw_data['user_phone'].apply(lambda x: filter(str.isdigit, x))
raw_data['user_phone_edited'] = raw_data['user_phone_edited'].apply(lambda x: "".join(x))

# Take only final 10 characters of phone number to remove any prefix digits
raw_data['user_phone_edited'] = raw_data['user_phone_edited'].apply(lambda x: x[-10:])

raw_data.head(10)

,transaction_id,created_at_UTC,user_phone,amount_paid_cents,zip_code,spot_id,year,month,year_month,user_phone_edited
0,1,2022-02-03 08:59:46,(674) 384-2776,9 065,30006,5,2022,02,202202,6743842776
1,2,2021-09-02 15:55:02,1-251-843-1844,6 844,23841,5,2021,09,202109,2518431844
2,3,2022-05-11 11:04:50,1-451-684-9420,9 446,482219,9,2022,05,202205,4516849420
3,4,2021-01-24 17:05:12,1-658-836-8233,660,15537,8,2021,01,202101,6588368233
4,5,2020-11-22 08:13:02,(481) 946-2170,5 714,221573,11,2020,11,202011,4819462170
5,6,2021-08-27 16:24:06,(771) 647-7413,8 822,66352,3,2021,08,202108,7716477413
6,7,2022-10-10 13:38:37,1-928-938-1403,4 526,354135,5,2022,10,202210,9289381403
7,8,2022-07-18 02:22:57,1-903-517-7431,5 284,563958,8,2022,07,202207,9035177431
8,9,2021-08-26 14:12:47,(774) 472-3583,3 413,41744-248,11,2021,08,202108,7744723583
9,10,2021-12-17 18:35:16,(392) 515-2956,182,38402,9,2021,12,202112,3925152956


Next, the amount_paid_cents field needs to be cleaned then converted to a numeric field.

In [7]:
# Replace the empty space within data points then convert to numeric type for future calculations
raw_data['amount_paid_cents_edited'] = raw_data['amount_paid_cents'].str.replace(' ', '',regex=True)
raw_data['amount_paid_cents_edited'] = pd.to_numeric(raw_data['amount_paid_cents_edited'])

# Convert cents to dollars for better interpretability
raw_data['amount_paid_dollars'] = raw_data['amount_paid_cents_edited']/100

raw_data.head(10)

,transaction_id,created_at_UTC,user_phone,amount_paid_cents,zip_code,spot_id,year,month,year_month,user_phone_edited,amount_paid_cents_edited,amount_paid_dollars
0,1,2022-02-03 08:59:46,(674) 384-2776,9 065,30006,5,2022,02,202202,6743842776,9065,90.65
1,2,2021-09-02 15:55:02,1-251-843-1844,6 844,23841,5,2021,09,202109,2518431844,6844,68.44
2,3,2022-05-11 11:04:50,1-451-684-9420,9 446,482219,9,2022,05,202205,4516849420,9446,94.46
3,4,2021-01-24 17:05:12,1-658-836-8233,660,15537,8,2021,01,202101,6588368233,660,6.60
4,5,2020-11-22 08:13:02,(481) 946-2170,5 714,221573,11,2020,11,202011,4819462170,5714,57.14
5,6,2021-08-27 16:24:06,(771) 647-7413,8 822,66352,3,2021,08,202108,7716477413,8822,88.22
6,7,2022-10-10 13:38:37,1-928-938-1403,4 526,354135,5,2022,10,202210,9289381403,4526,45.26
7,8,2022-07-18 02:22:57,1-903-517-7431,5 284,563958,8,2022,07,202207,9035177431,5284,52.84
8,9,2021-08-26 14:12:47,(774) 472-3583,3 413,41744-248,11,2021,08,202108,7744723583,3413,34.13
9,10,2021-12-17 18:35:16,(392) 515-2956,182,38402,9,2021,12,202112,3925152956,182,1.82


Our KPIs will not make use of the zip_code field, so we will leave this field un-cleaned for now.

Finally, we can drop all columns not necessary for our analysis.

In [8]:
data = raw_data[['transaction_id', 'created_at_UTC', 'year_month', 'user_phone_edited', 'amount_paid_dollars', 'spot_id']]
data.head(10)

,transaction_id,created_at_UTC,year_month,user_phone_edited,amount_paid_dollars,spot_id
0,1,2022-02-03 08:59:46,202202,6743842776,90.65,5
1,2,2021-09-02 15:55:02,202109,2518431844,68.44,5
2,3,2022-05-11 11:04:50,202205,4516849420,94.46,9
3,4,2021-01-24 17:05:12,202101,6588368233,6.60,8
4,5,2020-11-22 08:13:02,202011,4819462170,57.14,11
5,6,2021-08-27 16:24:06,202108,7716477413,88.22,3
6,7,2022-10-10 13:38:37,202210,9289381403,45.26,5
7,8,2022-07-18 02:22:57,202207,9035177431,52.84,8
8,9,2021-08-26 14:12:47,202108,7744723583,34.13,11
9,10,2021-12-17 18:35:16,202112,3925152956,1.82,9


### Data analysis

Below, I conducted a check to see how many users conducted transactions. I discovered that no user in the dataset performs more than 1 transaction, which means any metric on number of new users or average transaction spend by user would be meaningless metrics in this case.

In [9]:
# How many distinct users are in the dataset?
len(set(data['user_phone_edited'])) == len(set(data['transaction_id']))

True

In [10]:
# Aggregate number of transactions and total spend by month
agg_data = data.groupby(['year_month']).agg({'transaction_id':'count', 'amount_paid_dollars': 'sum'}).reset_index()
agg_data.columns = ['year_month', 'num_transactions', 'total_spend']

# Calculate average spend per transaction
agg_data['avg_spend_per_trans'] = round(agg_data['total_spend'] / agg_data['num_transactions'], 2)

agg_data.head(10)

,year_month,num_transactions,total_spend,avg_spend_per_trans
0,202011,16,792.18,49.51
1,202012,19,889.59,46.82
2,202101,32,1770.85,55.34
3,202102,14,693.40,49.53
4,202103,24,1049.14,43.71
5,202104,16,680.70,42.54
6,202105,22,1058.14,48.10
7,202106,22,1078.29,49.01
8,202107,16,920.02,57.50
9,202108,22,1093.85,49.72


In [11]:
# Smooth the dataset by generating a 3-month average
agg_data['num_transactions_3ma'] = agg_data.iloc[:,1].rolling(window=3).mean()
agg_data['total_spend_3ma'] = agg_data.iloc[:,2].rolling(window=3).mean()
agg_data['avg_spend_per_trans_3ma'] = agg_data.iloc[:,3].rolling(window=3).mean()

# Measure growth by calculating the percent change in the 3-month averages
agg_data['num_transactions_3ma_growth'] = agg_data['num_transactions_3ma'].diff()
agg_data['total_spend_3ma_growth'] = agg_data['total_spend_3ma'].diff()
agg_data['avg_spend_per_trans_3ma_growth'] = agg_data['avg_spend_per_trans_3ma'].diff()

agg_data['num_transactions_3ma_growth_perc'] = round(agg_data['num_transactions_3ma_growth'] / agg_data['num_transactions_3ma'], 3)
agg_data['total_spend_3ma_growth_perc'] = round(agg_data['total_spend_3ma_growth'] / agg_data['total_spend_3ma'], 3)
agg_data['avg_spend_per_trans_3ma_growth_perc'] = round(agg_data['avg_spend_per_trans_3ma_growth'] / agg_data['avg_spend_per_trans_3ma'], 3)

agg_data = agg_data[['year_month', 'num_transactions_3ma', 'num_transactions_3ma_growth_perc', 'total_spend_3ma', 'total_spend_3ma_growth_perc', 'avg_spend_per_trans_3ma', 'avg_spend_per_trans_3ma_growth_perc']]

agg_data.head(10)


,year_month,num_transactions_3ma,num_transactions_3ma_growth_perc,total_spend_3ma,total_spend_3ma_growth_perc,avg_spend_per_trans_3ma,avg_spend_per_trans_3ma_growth_perc
0,202011,NaN,NaN,NaN,NaN,NaN,NaN
1,202012,NaN,NaN,NaN,NaN,NaN,NaN
2,202101,22.333333,NaN,1150.873333,NaN,50.556667,NaN
3,202102,21.666667,-0.031,1117.946667,-0.029,50.563333,0.000
4,202103,23.333333,0.071,1171.130000,0.045,49.526667,-0.021
5,202104,18.000000,-0.296,807.746667,-0.450,45.260000,-0.094
6,202105,20.666667,0.129,929.326667,0.131,44.783333,-0.011
7,202106,20.000000,-0.033,939.043333,0.010,46.550000,0.038
8,202107,20.000000,0.000,1018.816667,0.078,51.536667,0.097
9,202108,20.000000,0.000,1030.720000,0.012,52.076667,0.010


We will now construct a dataset based on an aggregation of each spot by month. To do so, we will jump back to the 'data' dataframe created in the previous data prep step.

In [12]:
# Unique number of spots
len(set(data['spot_id']))

15

In [13]:
# Aggregate number of transactions and total spend by month and spot id
spot_data = data.groupby(['year_month', 'spot_id']).agg({'transaction_id':'count', 'amount_paid_dollars': 'sum'}).reset_index()
spot_data.columns = ['year_month', 'spot_id', 'num_transactions', 'total_spend']

# Calculate average spend per transaction
spot_data['avg_spend_per_trans'] = round(spot_data['total_spend'] / spot_data['num_transactions'], 2)

spot_data.head(10)

,year_month,spot_id,num_transactions,total_spend,avg_spend_per_trans
0,202011,1,1,12.40,12.40
1,202011,2,1,42.80,42.80
2,202011,3,1,74.12,74.12
3,202011,4,2,75.09,37.54
4,202011,6,1,52.61,52.61
5,202011,7,2,71.04,35.52
6,202011,8,3,142.48,47.49
7,202011,11,2,149.69,74.84
8,202011,13,1,63.10,63.10
9,202011,14,2,108.85,54.42


In [14]:
spot_data = spot_data.sort_values(['spot_id', 'year_month'])

spot_data['num_transactions_change'] = spot_data['num_transactions'].diff()
spot_data['total_spend_change'] = spot_data['total_spend'].diff()
spot_data['avg_spend_per_trans_change'] = spot_data['avg_spend_per_trans'].diff()

spot_data.head(10)

,year_month,spot_id,num_transactions,total_spend,avg_spend_per_trans,num_transactions_change,total_spend_change,avg_spend_per_trans_change
0,202011,1,1,12.40,12.40,NaN,NaN,NaN
10,202012,1,1,24.24,24.24,0.0,11.84,11.84
21,202101,1,3,207.83,69.28,2.0,183.59,45.04
41,202103,1,1,3.04,3.04,-2.0,-204.79,-66.24
76,202106,1,1,29.66,29.66,0.0,26.62,26.62
87,202107,1,2,92.44,46.22,1.0,62.78,16.56
97,202108,1,1,69.81,69.81,-1.0,-22.63,23.59
109,202109,1,1,38.31,38.31,0.0,-31.50,-31.50
121,202110,1,1,21.55,21.55,0.0,-16.76,-16.76
131,202111,1,1,11.53,11.53,0.0,-10.02,-10.02


In [15]:
# Identify areas of overlap when the previous record represents a different spot_id
mask = spot_data['spot_id'] != spot_data['spot_id'].shift(1)

# For records that match the above criteria, replace data point with a NaN
spot_data['num_transactions_change'][mask] = np.nan
spot_data['total_spend_change'][mask] = np.nan
spot_data['avg_spend_per_trans_change'][mask] = np.nan

# Clean up dataset by reorganizing columns
spot_data = spot_data[['year_month', 'spot_id', 'total_spend', 'total_spend_change', 'avg_spend_per_trans', 'avg_spend_per_trans_change', 'num_transactions', 'num_transactions_change']]
spot_data = spot_data.sort_values(by='year_month', ascending=False)

spot_data.head(10)

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,year_month,spot_id,total_spend,total_spend_change,avg_spend_per_trans,avg_spend_per_trans_change,num_transactions,num_transactions_change
265,202211,4,13.21,-118.46,13.21,-52.62,1,-1.0
268,202211,9,63.58,38.31,31.79,6.52,2,1.0
266,202211,5,40.96,-145.67,40.96,-5.70,1,-3.0
267,202211,7,30.23,11.05,30.23,11.05,1,0.0
269,202211,10,40.12,-48.02,40.12,-48.02,1,0.0
264,202210,15,85.91,-12.79,85.91,-12.79,1,0.0
252,202210,1,78.58,44.40,78.58,44.40,1,0.0
255,202210,4,131.67,58.30,65.83,-7.54,2,1.0
262,202210,12,9.24,-100.38,9.24,-45.57,1,-1.0
256,202210,5,186.63,150.86,46.66,10.89,4,3.0


### Data presentation

Output new prepared data files as JSON for ingestion in Observable.

In [16]:
# Convert DataFrames to JSON format
data.to_json('transaction_data.json', orient='records')
agg_data.to_json('revenue_metrics.json', orient='records')
spot_data.to_json('spot_metrics.json', orient='records')

A web-based dashboard has been built in Observable using JavaScript and D3.